In [3]:
import json
import random
import itertools
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Create outputs directory
outputs_dir = r"D://pythonprojects/game_math/slot-game-project-1/outputs"

# Game configuration
config = {
    "credits_start": 100,
    "spins_per_round": 8,
    "max_rounds": 3,
    "initial_bar_target": 1.0,
    "reels": {
        "rows": 1,
        "cols": 3,
        "symbols": ["A", "B", "C", "D", "E"],
        "multipliers": {
            "A": 2, "B": 3, "C": 4, "D": 5, "E": 6
        },
        "probabilities": {
            "A": 0.2, "B": 0.2, "C": 0.2, "D": 0.2, "E": 0.2
        }
    },
    "bar_fill_per_match": {
        "3_same": 0.20,
        "2_same": 0.067,
        "1_same": 0
    },
    "bar_bonus_multiplier": 2.0,
    "upgrades": {
        "reel_bias": {
            "cost": 50,
            "effect": "increase the probability of D by 5% and E by 10%, decrease the probability of A by 10% and B by 5%"
        },
        "extra_spins": {
            "cost": 50,
            "effect": "+2 spins per round"
        },
        "bonus_multiplier_upgrade": {
            "cost": 50,
            "effect": "increase all symbol multipliers by 1.0"
        }
    }
}

print("Game Configuration Loaded")
print(f"Bar targets: Round 1: {config['initial_bar_target']}, Round 2: {config['initial_bar_target'] + 0.5}, Round 3: {config['initial_bar_target'] + 1.0}")


Game Configuration Loaded
Bar targets: Round 1: 1.0, Round 2: 1.5, Round 3: 2.0


In [4]:
#engine.py

class SlotGame:
    def __init__(self, config_path=None, config_dict=None):
        if config_dict is not None:
            self.config = config_dict
        elif config_path is not None:
            with open(config_path, "r") as f:
                self.config = json.load(f)
        else:
            raise ValueError("Must provide either config_path or config_dict")

        self._validate_config()
        self.credits = float(self.config.get("credits_start", 100))
        self.spins_per_round = int(self.config.get("spins_per_round", 8))

        reels_config = self.config["reels"]
        self.rows = int(reels_config.get("rows", 1))
        self.cols = int(reels_config.get("cols", 3))
        self.symbols = list(reels_config["symbols"])
        self.base_multipliers = {s: float(v) for s, v in reels_config["multipliers"].items()}
        
        # Create normalized base probabilities
        raw_probs = {s: float(reels_config["probabilities"].get(s, 0.0)) for s in self.symbols}
        self.base_symbol_probs = self._normalize_probabilities(raw_probs)

        self.bar_fill_per_match = self.config.get(
            "bar_fill_per_match",
            {"3_same": 0.1, "2_same": 0.05, "1_same": 0}
        )
        self.base_bar_bonus_multiplier = float(self.config.get("bar_bonus_multiplier", 2.0))

        # Game state
        self.bar_progress = 0.0
        self.bar_target = float(self.config.get("initial_bar_target", 1.0))
        self.round = 1
        self.max_rounds = int(self.config.get("max_rounds", 3))

        # upgrades (all single-purchase)
        self.upgrades = {
            "reel_bias": False,
            "extra_spins": False,
            "bonus_multiplier_upgrade": False,
        }

        # Track per-round upgrade purchase restriction
        self.upgrade_bought_this_round = False

    def _validate_config(self):
        """Validate that config has all required keys and consistent data"""
        required_keys = ["reels"]
        for key in required_keys:
            if key not in self.config:
                raise ValueError(f"Missing required config key: {key}")
        
        reels_config = self.config["reels"]
        required_reel_keys = ["symbols", "multipliers", "probabilities"]
        for key in required_reel_keys:
            if key not in reels_config:
                raise ValueError(f"Missing required reels config key: {key}")

    def _normalize_probabilities(self, prob_dict):
        """Normalize probabilities to sum to 1.0"""
        total_prob = sum(prob_dict.values())
        if total_prob <= 0:
            raise ValueError("Symbol probabilities must sum to a positive number")
        return {s: prob_dict[s] / total_prob for s in prob_dict}

    def get_current_probabilities(self):
        """Get current symbol probabilities with all adjustments applied."""
        probs = self.base_symbol_probs.copy()
        
        # Apply reel_bias if active
        if self.upgrades.get("reel_bias"):
            if "D" in probs:
                probs["D"] += 0.05
            if "E" in probs:
                probs["E"] += 0.10
            if "A" in probs:
                probs["A"] = max(0.0, probs["A"] - 0.10)
            if "B" in probs:
                probs["B"] = max(0.0, probs["B"] - 0.05)
            
            # Renormalize after adjustments
            probs = self._normalize_probabilities(probs)
        
        return probs

    def get_effective_multipliers(self):
        """Get current multipliers with all upgrades applied"""
        if self.upgrades["bonus_multiplier_upgrade"]:
            return {s: val + 1.0 for s, val in self.base_multipliers.items()}
        return dict(self.base_multipliers)

    def spin_reels(self):
        """Spins each column independently using current probabilities."""
        current_probs = self.get_current_probabilities()
        symbols = list(current_probs.keys())
        weights = list(current_probs.values())
        return [random.choices(symbols, weights=weights, k=1)[0] for _ in range(self.cols)]

    def evaluate_spin(self, row):
        """Update the bar progress according to matches in this spin."""
        symbol_counts = {s: row.count(s) for s in self.symbols}
        bar_filled_this_spin = 0.0
        payout = 0.0
        matched = False

        multipliers = self.get_effective_multipliers()

        for symbol, count in symbol_counts.items():
            if count == 3:
                increment = multipliers[symbol] * float(self.bar_fill_per_match.get("3_same", 0.0))
                self.bar_progress += increment
                bar_filled_this_spin += increment
                matched = True
            elif count == 2:
                increment = multipliers[symbol] * float(self.bar_fill_per_match.get("2_same", 0.0))
                self.bar_progress += increment
                bar_filled_this_spin += increment
                matched = True

        if not matched:
            singles = [s for s, c in symbol_counts.items() if c == 1]
            if singles and float(self.bar_fill_per_match.get("1_same", 0.0)) > 0:
                best_symbol = max(singles, key=lambda s: multipliers[s])
                increment = multipliers[best_symbol] * float(self.bar_fill_per_match.get("1_same", 0.0))
                self.bar_progress += increment
                bar_filled_this_spin += increment

        if self.bar_progress > self.bar_target:
            self.bar_progress = self.bar_target

        return payout, bar_filled_this_spin

    def buy_upgrade(self, upgrade_name):
        """Buy an upgrade (for simulation)"""
        if self.upgrade_bought_this_round:
            return "You can only buy one upgrade per round."

        upgrade_config = self.config.get("upgrades", {})
        if upgrade_name not in upgrade_config:
            return f"Upgrade '{upgrade_name}' does not exist."
        if self.upgrades.get(upgrade_name):
            return f"Upgrade '{upgrade_name}' already purchased."

        cost = float(upgrade_config[upgrade_name]["cost"])
        if self.credits < cost:
            return f"Not enough credits to buy {upgrade_name}"

        self.credits -= cost
        self.upgrades[upgrade_name] = True
        self.upgrade_bought_this_round = True
        return f"Upgrade {upgrade_name} purchased successfully."

    def play_round_silent(self):
        """Plays one round silently and returns True if bonus triggered."""
        self.bar_progress = 0.0
        self.upgrade_bought_this_round = False

        spins_this_round = self.spins_per_round + (2 if self.upgrades["extra_spins"] else 0)

        for spin_num in range(1, spins_this_round + 1):
            row = self.spin_reels()
            payout, filled = self.evaluate_spin(row)

        bonus_triggered = False
        if self.bar_progress >= self.bar_target:
            multiplier = float(self.base_bar_bonus_multiplier)
            self.credits *= multiplier
            bonus_triggered = True

        return bonus_triggered


In [7]:
def generate_upgrade_strategies():
    """Generate all 13 possible upgrade strategies"""
    upgrades = ["reel_bias", "extra_spins", "bonus_multiplier_upgrade"]
    strategies = []

    # Skip-Skip: no upgrades purchased
    strategies.append(("skip", "skip"))

    # Skip in round 2, then buy any upgrade in round 3
    for u in upgrades:
        strategies.append(("skip", u))

    # Buy upgrade in round 2, then skip round 3
    for u in upgrades:
        strategies.append((u, "skip"))

    # Buy one upgrade in round 2, then buy a DIFFERENT upgrade in round 3
    for u1 in upgrades:
        for u2 in upgrades:
            if u1 != u2:
                strategies.append((u1, u2))

    return strategies

def format_strategy_name(strategy_tuple):
    """Convert strategy tuple to readable name"""
    round2, round3 = strategy_tuple
    
    if round2 == "skip" and round3 == "skip":
        return "No Upgrades"
    elif round2 == "skip":
        return f"R3: {round3}"
    elif round3 == "skip":
        return f"R2: {round2}"
    else:
        return f"R2: {round2} → R3: {round3}"

def format_upgrade_combo_name(upgrade_list):
    """Convert upgrade list to readable combination name"""
    if not upgrade_list or all(u == "skip" for u in upgrade_list):
        return "No Upgrades"
    
    active_upgrades = [u for u in upgrade_list if u != "skip"]
    if not active_upgrades:
        return "No Upgrades"
    
    return " + ".join(sorted(active_upgrades))

strategies = generate_upgrade_strategies()
print(f"Generated {len(strategies)} strategies:")
for i, strategy in enumerate(strategies[:13]):
    print(f"  {i+1}. {format_strategy_name(strategy)}")

Generated 13 strategies:
  1. No Upgrades
  2. R3: reel_bias
  3. R3: extra_spins
  4. R3: bonus_multiplier_upgrade
  5. R2: reel_bias
  6. R2: extra_spins
  7. R2: bonus_multiplier_upgrade
  8. R2: reel_bias → R3: extra_spins
  9. R2: reel_bias → R3: bonus_multiplier_upgrade
  10. R2: extra_spins → R3: reel_bias
  11. R2: extra_spins → R3: bonus_multiplier_upgrade
  12. R2: bonus_multiplier_upgrade → R3: reel_bias
  13. R2: bonus_multiplier_upgrade → R3: extra_spins


In [ ]:
def bar_fill_distribution(game: SlotGame):
    """Returns probability distribution of bar fill increments per spin"""
    symbols = game.symbols
    multipliers = game.get_effective_multipliers()
    current_probs = game.get_current_probabilities()

    values = []
    probs = []

    for row in itertools.product(symbols, repeat=game.cols):
        # probability of this row
        p_row = 1.0
        for sym in row:
            p_row *= current_probs[sym]

        symbol_counts = {s: row.count(s) for s in symbols}
        increment = 0.0

        for symbol, count in symbol_counts.items():
            if count == 3:
                increment += multipliers[symbol] * game.bar_fill_per_match.get("3_same", 0.0)
            elif count == 2:
                increment += multipliers[symbol] * game.bar_fill_per_match.get("2_same", 0.0)

        if increment == 0.0 and game.bar_fill_per_match.get("1_same", 0.0) > 0:
            singles = [s for s, c in symbol_counts.items() if c == 1]
            if singles:
                best_symbol = max(singles, key=lambda s: multipliers[s])
                increment = multipliers[best_symbol] * game.bar_fill_per_match.get("1_same", 0.0)

        values.append(increment)
        probs.append(p_row)

    return np.array(values), np.array(probs)

def expected_stats(game: SlotGame):
    """Compute expected bar fill and its stddev per spin."""
    values, probs = bar_fill_distribution(game)
    exp = np.sum(values * probs)
    exp_sq = np.sum((values ** 2) * probs)
    var = exp_sq - exp ** 2
    std = np.sqrt(max(var, 0.0))
    return exp, std

def generate_strategy_summary(config):
    """Generate summary statistics for all upgrade strategies."""
    strategies = generate_upgrade_strategies()
    summary_rows = []
    bar_targets = [1.0, 1.5, 2.0]

    for strategy_tuple in strategies:
        # Create fresh game for each strategy
        game = SlotGame(config_dict=config)
        
        # Apply upgrades (ignoring costs for theoretical analysis)
        round2_upgrade, round3_upgrade = strategy_tuple
        if round2_upgrade != "skip":
            game.upgrades[round2_upgrade] = True
        if round3_upgrade != "skip":
            game.upgrades[round3_upgrade] = True

        # Calculate statistics
        exp_fill, std_fill = expected_stats(game)
        
        # Get additional info
        current_probs = game.get_current_probabilities()
        multipliers = game.get_effective_multipliers()
        
        # Calculate spins per round with upgrades
        spins_per_round = game.spins_per_round + (2 if game.upgrades["extra_spins"] else 0)
        expected_per_round = exp_fill * spins_per_round
        
        # Check viability
        can_beat_r1 = expected_per_round >= bar_targets[0]
        can_beat_r2 = expected_per_round >= bar_targets[1] 
        can_beat_r3 = expected_per_round >= bar_targets[2]
        is_viable = can_beat_r1 and can_beat_r2 and can_beat_r3
        
        summary_rows.append({
            "Strategy": format_strategy_name(strategy_tuple),
            "Round_2_Upgrade": round2_upgrade,
            "Round_3_Upgrade": round3_upgrade,
            "Upgrade_Combination": format_upgrade_combo_name(list(strategy_tuple)),
            "Expected_Fill_per_Spin": exp_fill,
            "Std_Dev_Fill": std_fill,
            "Spins_per_Round": spins_per_round,
            "Expected_Fill_per_Round": expected_per_round,
            "Can_Beat_R1": can_beat_r1,
            "Can_Beat_R2": can_beat_r2,
            "Can_Beat_R3": can_beat_r3,
            "Is_Viable": is_viable,
            "Has_Reel_Bias": game.upgrades["reel_bias"],
            "Has_Extra_Spins": game.upgrades["extra_spins"],
            "Has_Bonus_Multiplier": game.upgrades["bonus_multiplier_upgrade"],
            "Avg_Symbol_Multiplier": sum(multipliers.values()) / len(multipliers),
            "High_Value_Prob_Sum": current_probs.get("D", 0) + current_probs.get("E", 0)})

    return pd.DataFrame(summary_rows)

print("Generating statistical analysis...")
stats_df = generate_strategy_summary(config)
print(f"Analysis complete. Found {stats_df['Is_Viable'].sum()} viable strategies out of {len(stats_df)}.")

print("\nViable Strategies:")
viable_strategies = stats_df[stats_df['Is_Viable']].copy()
for _, row in viable_strategies.iterrows():
    print(f"  {row['Strategy']}: Expected {row['Expected_Fill_per_Round']:.3f} per round")

Generating statistical analysis...
Analysis complete. Found 6 viable strategies out of 13.

Viable Strategies:
  R2: reel_bias → R3: extra_spins: Expected 2.208 per round
  R2: reel_bias → R3: bonus_multiplier_upgrade: Expected 2.128 per round
  R2: extra_spins → R3: reel_bias: Expected 2.208 per round
  R2: extra_spins → R3: bonus_multiplier_upgrade: Expected 2.008 per round
  R2: bonus_multiplier_upgrade → R3: reel_bias: Expected 2.128 per round
  R2: bonus_multiplier_upgrade → R3: extra_spins: Expected 2.008 per round


In [ ]:
def run_single_game(config_dict, strategy_tuple, verbose=False):
    """Run a single game following a fixed strategy."""
    game = SlotGame(config_dict=config_dict)
    total_spins = 0
    total_bar_progress = 0
    rounds_played = 0
    upgrade_costs_paid = 0
    upgrades_purchased = []
    round_results = []

    while game.round <= game.max_rounds:
        round_start_credits = game.credits
        spins_this_round = game.spins_per_round + (2 if game.upgrades["extra_spins"] else 0)
        
        # Strategy purchase logic: BEFORE playing the round, on rounds 2 and 3
        upgrade_bought_this_round = None
        upgrade_cost_this_round = 0
        
        if game.round in (2, 3):
            choice = strategy_tuple[game.round - 2]  # 0 for round 2, 1 for round 3
            if choice != "skip" and not game.upgrades.get(choice, False):
                # Check if we can afford the upgrade
                upgrade_config = config_dict.get("upgrades", {})
                if choice in upgrade_config:
                    cost = float(upgrade_config[choice]["cost"])
                    if game.credits >= cost:
                        result = game.buy_upgrade(choice)
                        if "purchased successfully" in result:
                            upgrade_bought_this_round = choice
                            upgrade_cost_this_round = cost
                            upgrade_costs_paid += cost
                            upgrades_purchased.append(choice)

        # Play the round
        round_bar_start = game.bar_progress
        bonus = game.play_round_silent()
        round_bar_end = game.bar_progress
        
        total_spins += spins_this_round
        total_bar_progress += round_bar_end
        rounds_played = game.round
        
        # Record round results
        round_results.append({
            "round": game.round,
            "bar_target": game.bar_target,
            "bar_progress": round_bar_end,
            "bonus_triggered": bonus,
            "spins": spins_this_round,
            "credits_start": round_start_credits,
            "credits_end": game.credits,
            "upgrade_bought": upgrade_bought_this_round,
            "upgrade_cost": upgrade_cost_this_round})

        if not bonus:
            break

        # Move to next round
        if game.round < game.max_rounds:
            game.bar_target += 0.5
            game.round += 1
        else:
            break

    return {
        "final_credits": game.credits,
        "rounds_played": rounds_played,
        "total_spins": total_spins,
        "total_bar_progress": total_bar_progress,
        "avg_bar_progress_per_round": total_bar_progress / rounds_played if rounds_played > 0 else 0,
        "upgrade_costs_paid": upgrade_costs_paid,
        "upgrades_purchased": upgrades_purchased,
        "round_results": round_results,
        "strategy": format_strategy_name(strategy_tuple),
        "upgrade_combination": format_upgrade_combo_name(list(strategy_tuple))}

def simulate_strategy(config_dict, strategy_tuple, num_runs=1000, seed=None):
    """Simulate a strategy multiple times and return summary statistics."""
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    results = []
    strategy_name = format_strategy_name(strategy_tuple)
    
    for i in range(num_runs):
        res = run_single_game(config_dict, strategy_tuple)
        results.append(res)

    df = pd.DataFrame(results)
    
    # Calculate statistics
    stats = {
        "strategy": strategy_name,
        "round2_upgrade": strategy_tuple[0],
        "round3_upgrade": strategy_tuple[1], 
        "upgrade_combination": format_upgrade_combo_name(list(strategy_tuple)),
        "num_runs": num_runs,
        
        # Credits statistics
        "avg_final_credits": df["final_credits"].mean(),
        "std_final_credits": df["final_credits"].std(),
        "median_final_credits": df["final_credits"].median(),
        "min_final_credits": df["final_credits"].min(),
        "max_final_credits": df["final_credits"].max(),
        
        # Performance statistics
        "avg_rounds_played": df["rounds_played"].mean(),
        "completion_rate": (df["rounds_played"] == 3).mean(),
        
        # Economic statistics
        "avg_upgrade_costs": df["upgrade_costs_paid"].mean(),
        "net_credits_gained": df["final_credits"].mean() - 100,
        "roi": (df["final_credits"].mean() - 100) / 100,
    }
    
    return stats, df

print("Running Monte Carlo simulations")
print("This may take a few minutes")

# Run simulations for all strategies
all_simulation_results = []
all_detailed_results = []

for i, strategy_tuple in enumerate(strategies):
    print(f"Simulating strategy {i+1}/{len(strategies)}: {format_strategy_name(strategy_tuple)}")
    
    summary_stats, detailed_df = simulate_strategy(
        config, 
        strategy_tuple, 
        num_runs=1000, 
        seed=42 + i)
    
    all_simulation_results.append(summary_stats)
    
    # Add strategy info to detailed results
    detailed_df["strategy"] = summary_stats["strategy"]
    detailed_df["strategy_tuple"] = str(strategy_tuple)
    all_detailed_results.append(detailed_df)

simulation_summary_df = pd.DataFrame(all_simulation_results)
detailed_simulation_df = pd.concat(all_detailed_results, ignore_index=True)


Running Monte Carlo simulations...
This may take a few minutes...
Simulating strategy 1/13: No Upgrades
Simulating strategy 2/13: R3: reel_bias
Simulating strategy 3/13: R3: extra_spins
Simulating strategy 4/13: R3: bonus_multiplier_upgrade
Simulating strategy 5/13: R2: reel_bias
Simulating strategy 6/13: R2: extra_spins
Simulating strategy 7/13: R2: bonus_multiplier_upgrade
Simulating strategy 8/13: R2: reel_bias → R3: extra_spins
Simulating strategy 9/13: R2: reel_bias → R3: bonus_multiplier_upgrade
Simulating strategy 10/13: R2: extra_spins → R3: reel_bias
Simulating strategy 11/13: R2: extra_spins → R3: bonus_multiplier_upgrade
Simulating strategy 12/13: R2: bonus_multiplier_upgrade → R3: reel_bias
Simulating strategy 13/13: R2: bonus_multiplier_upgrade → R3: extra_spins


In [ ]:
print(" STATISTICAL ANALYSIS RESULTS ")
print(f"Total strategies analyzed: {len(stats_df)}")
print(f"Viable strategies (theoretical): {stats_df['Is_Viable'].sum()}")

print("\nTop 5 strategies by expected fill per round:")
top_expected = stats_df.nlargest(5, "Expected_Fill_per_Round")[
    ["Strategy", "Expected_Fill_per_Round", "Is_Viable"]
]
print(top_expected.to_string(index=False))

print("\n" + "="*60)
print("=== SIMULATION RESULTS ===")
print(f"Strategies simulated: {len(simulation_summary_df)}")

print("\nTop 5 strategies by average final credits:")
top_credits = simulation_summary_df.nlargest(5, "avg_final_credits")[
    ["strategy", "avg_final_credits", "completion_rate", "roi"]
]
print(top_credits.to_string(index=False))

print("\nTop 5 strategies by completion rate:")
top_completion = simulation_summary_df.nlargest(5, "completion_rate")[
    ["strategy", "completion_rate", "avg_final_credits", "roi"]
]
print(top_completion.to_string(index=False))

print("\nTop 5 strategies by ROI:")
top_roi = simulation_summary_df.nlargest(5, "roi")[
    ["strategy", "roi", "avg_final_credits", "completion_rate"]
]
print(top_roi.to_string(index=False))

# Compare theoretical vs simulation results
print("\n" + "="*60)
print("=== THEORETICAL vs SIMULATION COMPARISON ===")

# Merge the datasets
comparison_df = stats_df.merge(
    simulation_summary_df[["strategy", "avg_final_credits", "completion_rate", "roi"]], 
    left_on="Strategy", 
    right_on="strategy", 
    how="left"
)

viable_comparison = comparison_df[comparison_df["Is_Viable"]].copy()
viable_comparison = viable_comparison.sort_values("Expected_Fill_per_Round", ascending=False)

print("Viable strategies - Theoretical vs Simulation:")
print("Strategy".ljust(30), "Expected Fill".ljust(15), "Completion Rate".ljust(15), "Avg Credits".ljust(12), "ROI")
print("-" * 80)

for _, row in viable_comparison.iterrows():
    strategy = row["Strategy"][:29]
    expected_fill = f"{row['Expected_Fill_per_Round']:.3f}"
    completion_rate = f"{row['completion_rate']:.1%}" if not pd.isna(row['completion_rate']) else "N/A"
    avg_credits = f"{row['avg_final_credits']:.0f}" if not pd.isna(row['avg_final_credits']) else "N/A"
    roi = f"{row['roi']:.1%}" if not pd.isna(row['roi']) else "N/A"
    
    print(f"{strategy.ljust(30)} {expected_fill.ljust(15)} {completion_rate.ljust(15)} {avg_credits.ljust(12)} {roi}")

=== STATISTICAL ANALYSIS RESULTS ===
Total strategies analyzed: 13
Viable strategies (theoretical): 6

Top 5 strategies by expected fill per round:
                                      Strategy  Expected_Fill_per_Round  Is_Viable
               R2: reel_bias → R3: extra_spins                 2.208158       True
               R2: extra_spins → R3: reel_bias                 2.208158       True
  R2: reel_bias → R3: bonus_multiplier_upgrade                 2.127886       True
  R2: bonus_multiplier_upgrade → R3: reel_bias                 2.127886       True
R2: extra_spins → R3: bonus_multiplier_upgrade                 2.008000       True

=== SIMULATION RESULTS ===
Strategies simulated: 13

Top 5 strategies by average final credits:
                                    strategy  avg_final_credits  completion_rate    roi
             R2: reel_bias → R3: extra_spins             233.95            0.401 1.3395
                               R2: reel_bias             232.25            0.393 

In [ ]:
# Save statistical analysis
stats_output_path = "D://pythonprojects/game_math/slot-game-project-1/outputs/statistical_analysis.xlsx"
with pd.ExcelWriter(stats_output_path, engine="openpyxl") as writer:
    stats_df.to_excel(writer, sheet_name="Strategy Analysis", index=False)
    
print(f"Statistical analysis saved to: {stats_output_path}")

# Save simulation results
simulation_output_path = "D://pythonprojects/game_math/slot-game-project-1/outputs/simulation_results.xlsx"
with pd.ExcelWriter(simulation_output_path, engine="openpyxl") as writer:
    # Summary sheet
    simulation_summary_df.to_excel(writer, sheet_name="Simulation Summary", index=False)
    
    # Sample detailed results (limit size)
    sample_detailed = detailed_simulation_df.sample(min(5000, len(detailed_simulation_df))) if len(detailed_simulation_df) > 5000 else detailed_simulation_df
    sample_detailed.to_excel(writer, sheet_name="Sample Detailed Results", index=False)

print(f"Simulation results saved to: {simulation_output_path}")

# Save combined analysis
combined_output_path = "D://pythonprojects/game_math/slot-game-project-1/outputs/combined_analysis.xlsx"
with pd.ExcelWriter(combined_output_path, engine="openpyxl") as writer:
    comparison_df.to_excel(writer, sheet_name="Combined Analysis", index=False)
    viable_comparison.to_excel(writer, sheet_name="Viable Strategies Only", index=False)

print(f"Combined analysis saved to: {combined_output_path}")

# Save game configuration
config_output_path = "D://pythonprojects/game_math/slot-game-project-1/outputs/game_config.json"
with open(config_output_path, "w") as f:
    json.dump(config, f, indent=2)

print(f"Game configuration saved to: {config_output_path}")


print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print(f"All results exported to the 'outputs' folder:")
print(f"- {stats_output_path}")
print(f"- {simulation_output_path}")
print(f"- {combined_output_path}")
print(f"- {config_output_path}")


Statistical analysis saved to: D://pythonprojects/game_math/slot-game-project-1/outputs/statistical_analysis.xlsx
Simulation results saved to: D://pythonprojects/game_math/slot-game-project-1/outputs/simulation_results.xlsx
Combined analysis saved to: D://pythonprojects/game_math/slot-game-project-1/outputs/combined_analysis.xlsx
Game configuration saved to: D://pythonprojects/game_math/slot-game-project-1/outputs/game_config.json

ANALYSIS COMPLETE!
All results exported to the 'outputs' folder:
- D://pythonprojects/game_math/slot-game-project-1/outputs/statistical_analysis.xlsx
- D://pythonprojects/game_math/slot-game-project-1/outputs/simulation_results.xlsx
- D://pythonprojects/game_math/slot-game-project-1/outputs/combined_analysis.xlsx
- D://pythonprojects/game_math/slot-game-project-1/outputs/game_config.json
